In [90]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [91]:
path = '/content/drive/MyDrive/Colab Notebooks/DACON CUP'

user =  pd.read_csv(path + '/open_data/info_user.csv', encoding = 'euc-kr')
login = pd.read_csv(path +'/open_data/info_login.csv', encoding = 'euc-kr')
competition = pd.read_csv(path +'/open_data/info_competition.csv', encoding = 'euc-kr')
info_submission =  pd.read_csv(path +'/open_data/info_submission.csv', encoding = 'euc-kr')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [192]:
train = pd.read_csv(path + '/open_data/train.csv', encoding = 'euc-kr')
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18


In [193]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19008 entries, 0 to 19007
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  19008 non-null  object
 1   사용자       19008 non-null  int64 
 2   세션        19008 non-null  int64 
 3   신규방문자     19008 non-null  int64 
 4   페이지뷰      19008 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 742.6+ KB


In [194]:
# date 기준으로.

train['DateTime'] = pd.to_datetime(train.DateTime) #dtype: datetime으로 변경
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()
train.head()

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845


In [195]:
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 30 # 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values


print('window_x.shape: ', window_x.shape)
print('window_y.shape: ', window_y.shape)

window_x.shape:  (732, 30, 4)
window_y.shape:  (732, 30, 4)


In [100]:
# window_x

- 시계열 데이터 임베딩 찾아서 적용해보기
- 스케일링 다르게 해보기
- dropout 적용해보기

In [199]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model


optimizer = Adam, loss = mse, activation_function = 따로지정x

In [200]:
model = Sequential()
model.add(LSTM(32, input_shape=(30, 4), return_sequences=True))  # ()
model.add(LSTM(32, return_sequences=True)) 
# model.add(LSTM(32, return_sequences=True)) 
model.add(TimeDistributed(Dense(4)))

model.summary()

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt)

model.fit(window_x, window_y, epochs=100, batch_size=30)

# lstm2층 - epochs100, batch_size=10

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 32)            4736      
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 32)            8320      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 4)             132       
Total params: 13,188
Trainable params: 13,188
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25/25 [==============================] - 4s 23ms/step - loss: 0.0217
Epoch 2/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0101
Epoch 3/100
25/25 [==============================] - 1s 23ms/step - loss: 0.0096
Epoch 4/100
25/25 [==============================] - 1s 25ms/step - loss: 0.0091
Epoch 5/100
25/25 [==============================] - 1s 

## test : last_month

In [212]:
# train은 이미 위에서 minmax scaler적용되어있음.

last_month = train.iloc[-30:,1:].values[np.newaxis,...] # np.newaxis: 3D로 차원을 늘려줌. 
last_month.shape

(1, 30, 4)

In [233]:
pred_month = model.predict(last_month)
pred_month[:5]

array([[[0.38988918, 0.38370723, 0.2831769 , 0.35649085],
        [0.42584968, 0.41972363, 0.31028152, 0.39880776],
        [0.4839755 , 0.48056078, 0.38403997, 0.42606884],
        [0.42508316, 0.4155653 , 0.30894116, 0.36046645],
        [0.3552722 , 0.3474655 , 0.2774096 , 0.25650254],
        [0.29504907, 0.28051865, 0.1924423 , 0.22581556],
        [0.3349247 , 0.3176093 , 0.21691819, 0.28028947],
        [0.43782055, 0.4194324 , 0.32147312, 0.35830352],
        [0.4307961 , 0.41820633, 0.30120403, 0.36840585],
        [0.39586526, 0.38768816, 0.3135556 , 0.3087331 ],
        [0.31007713, 0.30272764, 0.24639408, 0.23173746],
        [0.28607854, 0.28319347, 0.22286735, 0.19530392],
        [0.26119217, 0.25370237, 0.17054868, 0.18885493],
        [0.32039925, 0.3079184 , 0.20663486, 0.24411625],
        [0.5193938 , 0.50288975, 0.3978129 , 0.39398143],
        [0.5128634 , 0.50502926, 0.39631888, 0.39292422],
        [0.4583183 , 0.45703334, 0.3989874 , 0.36246622],
        [0.327

In [234]:
pred_month = pred_month.reshape(output_window,4)
pred_month = pred_month * size.values + mini.values
pred_month = pred_month.astype(int)
pred_month


array([[ 2113,  2072,   479, 55412],
       [ 2307,  2267,   525, 61989],
       [ 2622,  2595,   650, 66227],
       [ 2303,  2244,   523, 56030],
       [ 1925,  1876,   469, 39870],
       [ 1599,  1515,   326, 35101],
       [ 1815,  1715,   367, 43568],
       [ 2372,  2265,   544, 55694],
       [ 2334,  2258,   510, 57264],
       [ 2145,  2094,   530, 47989],
       [ 1680,  1635,   417, 36021],
       [ 1550,  1529,   377, 30358],
       [ 1415,  1370,   289, 29356],
       [ 1736,  1663,   350, 37945],
       [ 2814,  2716,   673, 61239],
       [ 2779,  2727,   670, 61075],
       [ 2483,  2468,   675, 56341],
       [ 1777,  1744,   412, 37239],
       [ 1627,  1596,   334, 34582],
       [ 1498,  1467,   276, 32420],
       [ 1616,  1597,   314, 34873],
       [ 2592,  2578,   585, 61798],
       [ 2637,  2590,   577, 64292],
       [ 2477,  2387,   551, 56554],
       [ 2283,  2255,   507, 49707],
       [ 2069,  2081,   456, 41702],
       [ 1940,  1954,   413, 42059],
 

In [235]:
pred_month.shape

(30, 4)

## submit

In [259]:
submission =  pd.read_csv(path + '/open_data/submission.csv', encoding = 'euc-kr')
submission.shape

(61, 5)

In [260]:
pred_month.shape

(30, 4)

In [247]:
pred_df = pd.DataFrame(pred_month)
pred_df.head()

,0,1,2,3
0,2113,2072,479,55412
1,2307,2267,525,61989
2,2622,2595,650,66227
3,2303,2244,523,56030
4,1925,1876,469,39870


In [277]:
# 30까지 모두 채우면 다시 0부터.

for i in range(len(submission)):
  submission['사용자'][i] = pred_df[0][i%30]
  submission['세션'][i] = pred_df[1][i%30]
  submission['신규방문자'][i] = pred_df[2][i%30]
  submission['페이지뷰'][i] = pred_df[3][i%30]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: Se

In [278]:
submission.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2113,2072,479,55412
1,2020-11-10,2307,2267,525,61989
2,2020-11-11,2622,2595,650,66227
3,2020-11-12,2303,2244,523,56030
4,2020-11-13,1925,1876,469,39870


In [279]:
submission.to_csv('lstm1.csv', index = False, encoding = 'euc-kr')

## 참조

In [ ]:
# 각 변수의 크기가 다르기 때문에 가중치를 부여한 RMSE로 모델의 성능을 평가

def dacon_rmse(true, pred):
    # true.shape // (N,4)
    # pred.shape // (N,4)
    # w0, w1, w2, w3 <= train.csv의 사용자 수, 세션 수, 신규 방문자 수, 페이지 뷰 수 4가지 항목별 평균값
    score = np.sqrt(np.mean(np.square(true[:,0] - pred[:,0]))) / w0 +\
            + np.sqrt(np.mean(np.square(true[:,1] - pred[:,1]))) / w1 +\
            + np.sqrt(np.mean(np.square(true[:,2] - pred[:,2]))) / w2 +\
            + np.sqrt(np.mean(np.square(true[:,3] - pred[:,3]))) / w3 +\
     return score

In [ ]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

train_sc

In [ ]:
# date 기준 변수 만들기

from datetime import datetime

def create_features(df):
    """
    Creates time series features from datetime index
    """
    # df['DateTime'] = df.index
    # df['hour'] = df['date'].dt.hour
    df['dayofweek'] = pd.to_datetime(df['date']).dt.dayofweek
    df['quarter'] = pd.to_datetime(df['date']).dt.quarter
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['dayofyear'] = pd.to_datetime(df['date']).dt.dayofyear
    df['dayofmonth'] = pd.to_datetime(df['date']).dt.day
    df['weekofyear'] = pd.to_datetime(df['date']).dt.weekofyear
    
    X = df[['사용자','세션','신규방문자','페이지뷰','date','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    X.set_index('date', inplace = True)
    X.index = pd.to_datetime(X.index)
    return X

## 예측값을 x로 다시 predict  
- 마이너스 값이 나와서 실패..

In [240]:
tr_month1 = pd.DataFrame(pred_month)

In [241]:
# 위 예측값을 x로 사용하기.
mini1 = tr_month1.iloc[:,1:].min()
size1 = tr_month1.iloc[:,1:].max() - tr_month1.iloc[:,1:].min()
tr_month1.iloc[:,1:] = (tr_month1.iloc[:,1:] -  mini1) / size1

tr_month1 = tr_month1.iloc[:,:].values[np.newaxis,...] # np.newaxis: 3D로 차원을 늘려줌. 
tr_month1.shape

(1, 30, 4)

In [242]:
pred_month1 = model.predict(tr_month1)
pred_month1[:5]

array([[[ 0.18093495,  0.15617856,  0.17545578,  0.07859027],
        [ 0.09990777,  0.06255519,  0.09158635, -0.03202646],
        [ 0.1238751 ,  0.08535202,  0.10137758, -0.0130481 ],
        [ 0.1609668 ,  0.12201263,  0.13407502,  0.02669705],
        [ 0.17643648,  0.13653305,  0.14765052,  0.04409892],
        [ 0.17911391,  0.13828306,  0.1470722 ,  0.04774056],
        [ 0.18096438,  0.13957691,  0.14543536,  0.05040792],
        [ 0.18431361,  0.14262158,  0.14594844,  0.05457786],
        [ 0.18804072,  0.14617258,  0.14746507,  0.05902395],
        [ 0.19144723,  0.14947562,  0.14904726,  0.06302298],
        [ 0.19444391,  0.15241605,  0.15047295,  0.06651456],
        [ 0.19708903,  0.15503576,  0.15175058,  0.0695814 ],
        [ 0.19941911,  0.15735956,  0.1528904 ,  0.07227365],
        [ 0.20145619,  0.1594011 ,  0.15389048,  0.07462133],
        [ 0.20322528,  0.16118066,  0.15475564,  0.07665686],
        [ 0.20475666,  0.1627254 ,  0.15549937,  0.07841668],
        

In [243]:
pred_month1 = pred_month1.reshape(output_window,4)
pred_month1 = pred_month1 * size.values + mini.values
pred_month1 = pred_month1.astype(int)
pred_month1


array([[  981,   844,   297, 12217],
       [  542,   338,   155, -4975],
       [  672,   461,   172, -2026],
       [  872,   659,   227,  4151],
       [  956,   738,   250,  6856],
       [  971,   747,   249,  7422],
       [  981,   754,   246,  7837],
       [  999,   771,   247,  8485],
       [ 1019,   790,   250,  9176],
       [ 1038,   808,   252,  9797],
       [ 1054,   823,   255, 10340],
       [ 1068,   838,   257, 10817],
       [ 1081,   850,   259, 11235],
       [ 1092,   861,   261, 11600],
       [ 1101,   871,   262, 11917],
       [ 1110,   879,   263, 12190],
       [ 1117,   886,   264, 12426],
       [ 1123,   893,   265, 12630],
       [ 1128,   898,   266, 12805],
       [ 1133,   903,   267, 12957],
       [ 1137,   907,   267, 13087],
       [ 1140,   910,   268, 13200],
       [ 1143,   913,   268, 13297],
       [ 1146,   916,   269, 13382],
       [ 1148,   918,   269, 13455],
       [ 1150,   920,   269, 13518],
       [ 1152,   922,   269, 13573],
 

#### 윈도우 크기 다르게 해서 train하고 test해보기

aa

## test = train[-61]

In [201]:
test = train[-61:]
test

,date,사용자,세션,신규방문자,페이지뷰
731,2020-09-09,0.497877,0.490461,0.355030,0.377532
732,2020-09-10,0.452464,0.450824,0.324260,0.339207
733,2020-09-11,0.405206,0.397111,0.278107,0.321836
734,2020-09-12,0.298320,0.295425,0.194083,0.242021
735,2020-09-13,0.342810,0.333951,0.198817,0.288169
...,...,...,...,...,...
787,2020-11-04,0.833487,0.828116,0.707101,0.724949
788,2020-11-05,0.766845,0.747546,0.617160,0.662015
789,2020-11-06,0.676020,0.662160,0.487574,0.566244
790,2020-11-07,0.456157,0.447490,0.313609,0.369188


In [202]:
# scaling

input_window = 30
output_window = 30

test_x = np.zeros((test.shape[0] - (input_window + output_window), input_window, 4))
test_y = np.zeros((test.shape[0] - (input_window + output_window), output_window, 4))

for start in range(test.shape[0] - (input_window + output_window)):
    end = start + input_window    
    test_x[start,:, :] = test.iloc[start : end                , 1: ].values
    test_y[start,:, :] = test.iloc[end   : end + output_window, 1: ].values


print('test_x.shape: ', test_x.shape)
print('test_y.shape: ', test_y.shape)

test_x.shape:  (1, 30, 4)
test_y.shape:  (1, 30, 4)


In [205]:
pred = model.predict(test_x)
pred[:5]

array([[[0.40022817, 0.39372706, 0.27692318, 0.33888835],
        [0.38961065, 0.3810348 , 0.25960314, 0.3113099 ],
        [0.4294502 , 0.41507578, 0.28880233, 0.3883028 ],
        [0.57350516, 0.56259114, 0.4501635 , 0.5218927 ],
        [0.4765348 , 0.467364  , 0.3573951 , 0.40904856],
        [0.4156783 , 0.4077924 , 0.30611813, 0.35120863],
        [0.39636064, 0.39218706, 0.30435696, 0.30528963],
        [0.36307785, 0.3584634 , 0.27156314, 0.28400347],
        [0.27903116, 0.27569184, 0.22178051, 0.19545761],
        [0.37462008, 0.3721298 , 0.3127888 , 0.26252687],
        [0.47159705, 0.46326333, 0.39487627, 0.3773024 ],
        [0.5758832 , 0.5726595 , 0.5498195 , 0.51826733],
        [0.4800876 , 0.47606337, 0.40594587, 0.41935587],
        [0.5029881 , 0.49729228, 0.44717255, 0.40659106],
        [0.41189307, 0.40996557, 0.32963312, 0.3260718 ],
        [0.32444862, 0.31973892, 0.24362595, 0.24606156],
        [0.31358963, 0.30821908, 0.22619447, 0.24227071],
        [0.573

In [206]:
pred = pred.reshape(output_window,4)
pred = pred * size.values + mini.values
pred = pred.astype(int)
pred

array([[  2169,   2126,    469,  52676],
       [  2111,   2058,    439,  48389],
       [  2327,   2241,    489,  60357],
       [  3107,   3038,    761,  81121],
       [  2582,   2524,    604,  63581],
       [  2252,   2202,    518,  54591],
       [  2148,   2118,    515,  47454],
       [  1967,   1936,    459,  44145],
       [  1512,   1489,    375,  30382],
       [  2030,   2010,    529,  40807],
       [  2555,   2502,    668,  58647],
       [  3120,   3092,    930,  80557],
       [  2601,   2571,    687,  65183],
       [  2725,   2685,    756,  63199],
       [  2232,   2214,    558,  50684],
       [  1758,   1727,    412,  38248],
       [  1699,   1665,    383,  37658],
       [  3105,   3034,    777,  77979],
       [  3238,   3187,    846,  80160],
       [  3406,   3323,    811,  87295],
       [  3934,   3844,    924, 103179],
       [  3397,   3309,    775,  89027],
       [  2084,   2102,    496,  55332],
       [  2265,   2250,    480,  51135],
       [  5027, 

In [207]:
pred.shape

(30, 4)

In [ ]:
baseline = pd.read_csv(path + '/open_data/baseline.csv', encoding = 'euc-kr' )
baseline.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3252,3263,777,88002
1,2020-11-10,2825,2758,671,73749
2,2020-11-11,2614,2588,599,67300
3,2020-11-12,2391,2373,557,63723
4,2020-11-13,2063,2047,467,54340


In [ ]:
last_month = train.iloc[-30:,1:].values[np.newaxis,...].astype(float) #float() # <- 수정 후
last_month

In [ ]:

#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후
# last_month = torch.tensor(last_month).float() # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model.predict(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후

    pred_week = next_week.reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission